Reconocimiento de patrones binarios usando neuronas de McCulloch-Pitts
===

* *30 min* | Última modificación: Marzo 30, 2020

En este tutorial se presenta la implementación de una red neuronal que permite reconocer el siguiente conjunto de patrones binarios.


    Entrada   Salida
    ------------------
       000       0
       001       1
       010       0
       011       0
       100       1
       101       0
       110       1
       111       1

       

## Implementación en TensorFlow

In [2]:
## 
## Preparación
##
import tensorflow as tf

##
## Peso de las neuronas inhibitorias
##
N = -10.0

## Pesos asociados a las conexiones. 
## Se traspone la matriz para que cada
## fila representa un patrón y cada columna
## un dígito binario    
weights = tf.constant([[N, N, 1],  # 001
                       [1, N, N],  # 100
                       [1, 1, N],  # 110
                       [1, 1, 1]], # 111 
                      tf.float32)


## valor límite de las neuronas para 
## la función de activación
threshold = tf.constant([1, 1, 2, 3],
                        tf.float32)

                       
                       
##
## vector fila que representa la entrada a la red neuronal
##
x = tf.placeholder(tf.float32, shape=(3,))



## 
## Cálculo de la salida de la red
##
out0 = tf.matmul(weights, tf.reshape(x, [3, 1]) )       # suma de las entradas 
out1 = tf.reshape(out0, [-1])
out2 = tf.subtract(out1, threshold) # resta el umbral: g() - umbral
out3 = tf.map_fn(lambda m: tf.cond(tf.greater_equal(m, 0.0), 
                                      true_fn  = lambda : tf.constant(1.0), 
                                      false_fn = lambda : tf.constant(0.0)),
                  out2)

##
## Computa la función OR
##
nnet = tf.reduce_max(out3) # salida de la red neuronal

##
## Abre una sesión y ejecuta
## los cálculos para diferentes valores de entrada
##
with tf.Session() as sess:

    print(sess.run(nnet, feed_dict = {x: [0,0,0]}))
    print(sess.run(nnet, feed_dict = {x: [0,0,1]}))
    print(sess.run(nnet, feed_dict = {x: [0,1,0]}))
    print(sess.run(nnet, feed_dict = {x: [0,1,1]}))
    print(sess.run(nnet, feed_dict = {x: [1,0,0]}))
    print(sess.run(nnet, feed_dict = {x: [1,0,1]}))
    print(sess.run(nnet, feed_dict = {x: [1,1,0]}))
    print(sess.run(nnet, feed_dict = {x: [1,1,1]}))


ModuleNotFoundError: No module named 'tensorflow'

## Apéndice --- Implementación en R

A continuación se presenta el programa en R para simular una red de neuronas de McCulloch-Pitts. 

In [4]:
%%R
nnet <- function(x) {

    ## 
    ## define x como un vector fila
    ##
    x <- matrix(x, nrow = 3, ncol = 1)
    
    ## Peso de las neuronas inhibitorias, es 
    ## un valor mucho más grande que la cantidad
    ## de entradas
    N <- -10.0

    ## Las conexiones son representadas mediante 
    ## una matriz donde las filas corresponden 
    ## a los patrones y las columnas a los bits.
    ## Note que N representa las conexiones inhibitorias.
    ## Las conexiones excitatorias tienen un 
    ## valor de 1.
    weights <- matrix(c(N, N, 1,  # 001
                        1, N, N,  # 100
                        1, 1, N,  # 110
                        1, 1, 1), # 111 
                      nrow = 3,   # número de bits por patron
                      ncol = 4)   # número de patrones
    
    ## se traspone la matriz para que cada
    ## fila representa un patrón y cada columna
    ## un dígito binario
    weights <- t(weights)
    
    ## El umbral de las neuronas de la primera capa
    ## se almacena en un vector
    threshold <- matrix(c(1, 1, 2, 3), nrow = 4,, ncol = 1)
    
    ## operaciones matemáticas
    out <- weights %*% x          # multiplicación matricial
    out <- out - threshold        # resta el umbral 
    out <- ifelse(out >= 0, 1, 0) # aplica la función f
    out <- max(out)               # computa la función OR
    
    return(out)                   # retorna la salida de la red neuronal 
}

## 
## Verificacion
##
print(nnet(c(0,0,0)) == 0)
print(nnet(c(0,0,1)) == 1)
print(nnet(c(0,1,0)) == 0)
print(nnet(c(0,1,1)) == 0)
print(nnet(c(1,0,0)) == 1)
print(nnet(c(1,0,1)) == 0)
print(nnet(c(1,1,0)) == 1)
print(nnet(c(1,1,1)) == 1)

[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE
[1] TRUE


## Apéndice --- Implementación en TensorFlow for R

In [ ]:
%%R
## 
## Preparación
##
library(tensorflow)

##
## Peso de las neuronas inhibitorias
##
N <- -10.0

## Pesos asociados a las conexiones. 
## Se traspone la matriz para que cada
## fila representa un patrón y cada columna
## un dígito binario    
weights <- tf$constant( t(matrix(c(N, N, 1,  # 001
                                   1, N, N,  # 100
                                   1, 1, N,  # 110
                                   1, 1, 1), # 111 
                                 nrow = 3,   # número de bits de la entrada
                                 ncol = 4)), # número de patrones)
                          tf$float32)


## valor límite de las neuronas para 
## la función de activación
threshold <- tf$constant(matrix(c(1, 1, 2, 3), 
                                nrow = 4, 
                                ncol = 1), 
                         tf$float32)

                       
                       
##
## vector fila que representa la entrada a la red neuronal
##
x <- tf$placeholder(tf$float32, shape=c(3, 1))



## 
## Cálculo de la salida de la red
##
out0 <- tf$matmul(weights, x)        # suma de las entradas 
out1 <- tf$subtract(out0, threshold) # resta el umbral: g() - umbral
out2 <- tf$map_fn(function(m) tf$cond(tf$greater_equal(m[[0]], 0.0), 
                                      true_fn = function() tf$constant(1.), 
                                      false_fn = function() tf$constant(0.0)),
                  out1)

##
## Computa la función OR
##
nnet <- tf$reduce_max(out2) # salida de la red neuronal

##
## Abre una sesión y ejecuta
## los cálculos para diferentes valores de entrada
##
with(tf$Session() %as% sess, {
    
    print(sess$run(nnet, feed_dict =dict(x = matrix(c(0,0,0), 3, 1))))
    print(sess$run(nnet, feed_dict =dict(x = matrix(c(0,0,1), 3, 1))))
    print(sess$run(nnet, feed_dict =dict(x = matrix(c(0,1,0), 3, 1))))
    print(sess$run(nnet, feed_dict =dict(x = matrix(c(0,1,1), 3, 1))))
    print(sess$run(nnet, feed_dict =dict(x = matrix(c(1,0,0), 3, 1))))
    print(sess$run(nnet, feed_dict =dict(x = matrix(c(1,0,1), 3, 1))))
    print(sess$run(nnet, feed_dict =dict(x = matrix(c(1,1,0), 3, 1))))
    print(sess$run(nnet, feed_dict =dict(x = matrix(c(1,1,1), 3, 1))))
})